In [72]:
import os
import numpy as np
import gymnasium as gym
import pandas as pd
import numpy as np
from finrl.main import check_and_make_directories
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
from pypfopt.efficient_frontier import EfficientFrontier
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
import matplotlib.pyplot as plt
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts


check_and_make_directories([TRAINED_MODEL_DIR])

In [2]:
# train = pd.read_csv('data/train.csv')
# train = train.set_index(train.columns[0])
# train.index.names = ['']

# trade = pd.read_csv('data/trade.csv')
# trade = trade.set_index(trade.columns[0])
# trade.index.names = ['']

In [3]:

train = pd.read_pickle('data/train.pickle')
train.index.names = ['']

trade = pd.read_pickle('data/trade.pickle')
trade.index.names = ['']

In [4]:
trade.head()

,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,cov_list,return_list
,,,,,,,,,,,,,,,,,,,,
0,2022-07-01,116.428902,116.966073,113.977456,116.018702,1119800,A,4,-1.243367,126.593228,106.836679,47.215885,-41.333695,3.151366,118.512244,119.273005,26.700001,704.943013,"[[0.00033644443233636076, 0.000219604528339706...",tic A AAL AAP ...
0,2022-07-01,13.130000,13.180000,12.520000,12.780000,34110400,AAL,4,-0.929487,16.782486,11.131514,42.042263,-77.882730,26.682855,14.950667,16.521667,26.700001,704.943013,"[[0.00033644443233636076, 0.000219604528339706...",tic A AAL AAP ...
0,2022-07-01,160.159119,163.867022,158.766356,159.937746,784400,AAP,4,-3.802957,176.454994,150.658122,41.489812,-60.850421,18.128097,166.314550,180.640219,26.700001,704.943013,"[[0.00033644443233636076, 0.000219604528339706...",tic A AAL AAP ...
0,2022-07-01,136.581299,136.689440,133.366591,133.740157,71051600,AAPL,4,-2.301428,147.113393,125.673664,45.052172,-36.439500,17.240089,138.153923,147.386981,26.700001,704.943013,"[[0.00033644443233636076, 0.000219604528339706...",tic A AAL AAP ...
0,2022-07-01,103.027710,103.261396,100.625413,101.148868,3291700,ABT,4,-1.119116,110.437231,92.840119,47.685122,-19.392709,4.354522,103.568615,105.811821,26.700001,704.943013,"[[0.00033644443233636076, 0.000219604528339706...",tic A AAL AAP ...


In [5]:
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 394, State Space: 3941


In [6]:
df_account_value_dt = pd.read_pickle('data/df_account_value_dt.pkl')
df_actions_dt = pd.read_pickle('data/df_actions_dt.pkl')

In [7]:
df_account_value_ppo = pd.read_pickle('data/df_account_value_ppo.pkl')
df_actions_ppo = pd.read_pickle('data/df_actions_ppo.pkl')

In [8]:
# Helps us process data into a form for weight calculation
def process_df_for_mvo(df):
  df = df.sort_values(['date','tic'],ignore_index=True)[['date','tic','close']]
  fst = df
  fst = fst.iloc[0:stock_dimension, :]
  tic = fst['tic'].tolist()

  mvo = pd.DataFrame()

  for k in range(len(tic)):
    mvo[tic[k]] = 0

  for i in range(df.shape[0]//stock_dimension):
    n = df
    n = n.iloc[i * stock_dimension:(i+1) * stock_dimension, :]
    date = n['date'][i*stock_dimension]
    mvo.loc[date] = n['close'].tolist()

  return mvo

# Calculates weights of average return and covariance matrix
def StockReturnsComputing(StockPrice, Rows, Columns):
  StockReturn = np.zeros([Rows-1, Columns])
  for j in range(Columns):        # j: Assets
    for i in range(Rows-1):     # i: Daily Prices
      StockReturn[i,j]=((StockPrice[i+1, j]-StockPrice[i,j])/StockPrice[i,j])* 100

  return StockReturn

In [9]:
StockData = process_df_for_mvo(train)
TradeData = process_df_for_mvo(trade)

TradeData.to_numpy()

C:\Users\drebi\AppData\Local\Temp\ipykernel_31132\294320812.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mvo[tic[k]] = 0
C:\Users\drebi\AppData\Local\Temp\ipykernel_31132\294320812.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mvo[tic[k]] = 0
C:\Users\drebi\AppData\Local\Temp\ipykernel_31132\294320812.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) i

array([[116.42890167,  13.13000011, 160.15911865, ..., 109.66519165,
        104.34970093,  45.35165024],
       [117.74740601,  13.88000011, 168.15600586, ..., 110.24777985,
        103.98005676,  45.08684921],
       [118.35294342,  13.89000034, 167.65792847, ..., 109.7779541 ,
        102.85163116,  44.4954567 ],
       ...,
       [134.80755615,  14.13000011,  72.26625824, ..., 137.4912262 ,
        118.10128021,  39.24219894],
       [136.16184998,  13.88000011,  72.68218994, ..., 137.69519043,
        117.70678711,  39.70812607],
       [137.99066162,  13.97999954,  72.95302582, ..., 139.0743866 ,
        120.03429413,  39.28974915]], shape=(459, 394))

In [10]:
TradeData.head()

,A,AAL,AAP,AAPL,ABT,ACN,ADBE,ADI,ADM,ADP,...,WU,WY,WYNN,XEL,XOM,XRAY,XRX,YUM,ZBH,ZION
2022-07-01,116.428902,13.13,160.159119,136.581299,103.027710,265.930206,368.480011,134.889359,69.337593,199.049957,...,12.820278,30.298096,56.667645,65.121376,78.262825,33.328892,11.309067,109.665192,104.349701,45.351650
2022-07-05,117.747406,13.88,168.156006,139.166824,102.167747,260.737610,376.489990,135.040375,65.680595,198.349808,...,12.789773,30.494257,56.287773,62.664642,75.813477,33.236816,11.196938,110.247780,103.980057,45.086849
2022-07-06,118.352943,13.89,167.657928,140.503830,101.597557,261.467255,382.829987,136.578888,65.535400,202.811951,...,12.820278,30.253510,54.388470,63.668858,74.445770,32.794888,10.964670,109.777954,102.851631,44.495457
2022-07-07,120.276993,14.10,169.705582,143.875854,102.345329,264.745758,390.890015,140.231705,67.513634,202.121155,...,12.774520,30.743919,56.355961,62.386696,76.823616,32.647587,11.333096,110.633011,103.172638,45.439915
2022-07-08,119.651924,13.97,168.700180,144.554184,102.130356,264.433105,389.440002,141.524811,66.551735,202.821274,...,12.621985,31.216482,54.992352,62.404621,76.948753,32.417404,11.517309,110.200790,102.472221,45.051544


In [11]:
initial_portfolio_value = 1000000

assets = trade['tic'].unique()
n = len(assets)

naive_weights = {asset: 1/n for asset in assets}

naive_weights_df = pd.DataFrame.from_dict(naive_weights, orient='index', columns=['Weight'])

print("Naive Strategy Weights:")
print(naive_weights_df)

trade_with_weights = trade.merge(naive_weights_df, left_on='tic', right_index=True)

trade_with_weights['1/n'] = trade_with_weights['close'] * trade_with_weights['Weight'] * initial_portfolio_value

portfolio_value = trade_with_weights.groupby('date')['1/n'].sum()

naive_result = portfolio_value / portfolio_value.iloc[0] * initial_portfolio_value

print("Portfolio Value Using Naive Strategy:")
print(naive_result)

Naive Strategy Weights:
        Weight
A     0.002538
AAL   0.002538
AAP   0.002538
AAPL  0.002538
ABT   0.002538
...        ...
XRAY  0.002538
XRX   0.002538
YUM   0.002538
ZBH   0.002538
ZION  0.002538

[394 rows x 1 columns]
Portfolio Value Using Naive Strategy:
date
2022-07-01    1.000000e+06
2022-07-05    9.967892e+05
2022-07-06    9.998956e+05
2022-07-07    1.011268e+06
2022-07-08    1.008707e+06
                  ...     
2024-04-23    1.307739e+06
2024-04-24    1.307598e+06
2024-04-25    1.302921e+06
2024-04-26    1.306068e+06
2024-04-29    1.313012e+06
Name: 1/n, Length: 459, dtype: float64


In [12]:
#compute asset returns
arStockPrices = np.asarray(StockData)
[Rows, Cols]=arStockPrices.shape
arReturns = StockReturnsComputing(arStockPrices, Rows, Cols)

#compute mean returns and variance covariance matrix of returns
meanReturns = np.mean(arReturns, axis = 0)
covReturns = np.cov(arReturns, rowvar=False)

#set precision for printing results
np.set_printoptions(precision=3, suppress = True)

#display mean returns and variance-covariance matrix of returns
print('Mean returns of assets in k-portfolio 1\n', meanReturns)
print('Variance-Covariance matrix of returns\n', covReturns)

Mean returns of assets in k-portfolio 1
 [0.072 0.088 0.067 0.113 0.064 0.079 0.092 0.075 0.052 0.074 0.087 0.061
 0.055 0.042 0.052 0.055 0.072 0.077 0.083 0.066 0.088 0.12  0.082 0.065
 0.092 0.129 0.074 0.041 0.066 0.089 0.074 0.109 0.077 0.084 0.026 0.057
 0.071 0.049 0.063 0.052 0.071 0.078 0.062 0.094 0.062 0.051 0.039 0.041
 0.059 0.052 0.018 0.072 0.036 0.088 0.056 0.059 0.077 0.063 0.049 0.036
 0.039 0.043 0.041 0.063 0.063 0.078 0.066 0.011 0.12  0.069 0.09  0.071
 0.038 0.081 0.075 0.037 0.046 0.062 0.068 0.064 0.111 0.072 0.067 0.115
 0.051 0.06  0.082 0.063 0.084 0.032 0.092 0.095 0.042 0.077 0.103 0.053
 0.052 0.05  0.047 0.068 0.04  0.078 0.043 0.088 0.099 0.051 0.06  0.091
 0.069 0.076 0.062 0.048 0.046 0.042 0.056 0.083 0.065 0.054 0.047 0.074
 0.044 0.094 0.062 0.045 0.062 0.082 0.054 0.059 0.061 0.071 0.037 0.059
 0.102 0.035 0.049 0.069 0.11  0.038 0.074 0.049 0.052 0.024 0.058 0.061
 0.074 0.026 0.069 0.038 0.003 0.057 0.01  0.057 0.044 0.066 0.043 0.076
 0.064 0.0

In [13]:
ef_mean = EfficientFrontier(meanReturns, covReturns, weight_bounds=(0, 0.5), solver='ECOS')
raw_weights_mean = ef_mean.max_sharpe()
cleaned_weights_mean = ef_mean.clean_weights()

# solver_options = {'verbose': True}
# raw_weights_mean = ef_mean.max_sharpe(solver='ECOS', solver_options=solver_options)
# Use shape of 'Last Price' in the range to avoid shape mismatch error
mvo_weights = np.array([1000000 * cleaned_weights_mean[i] for i in range(394)])

mvo_weights

array([     0.,      0.,      0.,  47830.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0., 105960.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,  38200.,      0.,      0.,      0.,      0.,      0.,
            0.,      0.,      0.,  33000.,      0.,      0.,      0.,
            0.,      0.,      0.,      0.,  40980.,      0.,      0.,
            0.,      0.,      0.,      0.,      0.,      0.,      0.,
            0.,     

In [14]:
mvo_weights.shape

(394,)

In [15]:
LastPrice = np.array([1/p for p in StockData.tail(1).to_numpy()[0]])

In [16]:
LastPrice.shape

(394,)

In [17]:
Initial_Portfolio = np.multiply(mvo_weights, LastPrice)

Initial_Portfolio

array([   0.   ,    0.   ,    0.   ,  355.855,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,   49.304,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,  428.506,
          0.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   , 1262.182,    0.   ,    0.   ,    0.   ,
          0.   ,    0.   ,    0.   ,    0.   ,   89.275,    0.

In [18]:
Portfolio_Assets = TradeData @ Initial_Portfolio
MVO_result = pd.DataFrame(Portfolio_Assets, columns=["Mean Var"])

MVO_result

,Mean Var
2022-07-01,1.011971e+06
2022-07-05,1.012800e+06
2022-07-06,1.019810e+06
2022-07-07,1.026348e+06
2022-07-08,1.027735e+06
...,...
2024-04-23,1.448700e+06
2024-04-24,1.441547e+06
2024-04-25,1.443845e+06
2024-04-26,1.446996e+06


In [19]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2022-07-01'
TRADE_START_DATE = '2022-07-01'
TRADE_END_DATE = '2024-05-01'


In [20]:
df_sp = YahooDownloader(start_date = TRADE_START_DATE,
                        end_date = TRADE_END_DATE,
                        ticker_list = ['^GSPC']).fetch_data()

[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (460, 8)


In [21]:
df_sp = df_sp[['date','close']]
fst_day = df_sp['close'][0]
sp = pd.merge(df_sp['date'], df_sp['close'].div(fst_day).mul(1000000),
               how='outer', left_index=True, right_index=True).set_index('date')

In [22]:
df_result_dt = df_account_value_dt.set_index(df_account_value_dt.columns[0]) #if if_using_a2c else None

result = pd.DataFrame()
dt_result = pd.merge(result, df_result_dt, how='outer', left_index=True, right_index=True)

In [23]:
col_name = []
col_name.append('DT')
dt_result.columns = col_name

In [24]:
df_result_ppo = df_account_value_ppo.set_index(df_account_value_ppo.columns[0]) #if if_using_a2c else None

result = pd.DataFrame()
result = pd.merge(result, df_result_ppo, how='outer', left_index=True, right_index=True)

In [25]:
col_name = []
col_name.append('PPO')
result.columns = col_name

In [26]:
result = pd.merge(result, dt_result, how='outer', left_index=True, right_index=True)
result = pd.merge(result, MVO_result, how='outer', left_index=True, right_index=True)
result = pd.merge(result, naive_result, how='outer', left_index=True, right_index=True)
result = pd.merge(result, sp, how='outer', left_index=True, right_index=True).fillna(method='bfill')


C:\Users\drebi\AppData\Local\Temp\ipykernel_31132\3016569050.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  result = pd.merge(result, sp, how='outer', left_index=True, right_index=True).fillna(method='bfill')


In [27]:
result.rename(columns={'close':'S&P500'}, inplace=True)


In [28]:
result

,PPO,DT,Mean Var,1/n,S&P500
date,,,,,
2022-07-01,1.000000e+06,1.000000e+06,1.011971e+06,1.000000e+06,1.000000e+06
2022-07-05,9.987264e+05,9.986497e+05,1.012800e+06,9.967892e+05,1.001584e+06
2022-07-06,9.992308e+05,9.992926e+05,1.019810e+06,9.998956e+05,1.005163e+06
2022-07-07,1.004523e+06,1.004898e+06,1.026348e+06,1.011268e+06,1.020205e+06
2022-07-08,1.001750e+06,1.002063e+06,1.027735e+06,1.008707e+06,1.019358e+06
...,...,...,...,...,...
2024-04-24,1.495596e+06,1.285611e+06,1.441547e+06,1.307598e+06,1.325802e+06
2024-04-25,1.495356e+06,1.281273e+06,1.443845e+06,1.302921e+06,1.319734e+06
2024-04-26,1.495850e+06,1.283513e+06,1.446996e+06,1.306068e+06,1.333208e+06


In [32]:
plt.rcParams["figure.figsize"] = (15,5)
ax = result[['DT', '1/n']].plot()
plt.title("DT vs Naive Account Value Over Time")
plt.xlabel("date")
plt.ylabel("Account Value")
plt.legend(["DT", "Naive"])
plt.grid(True)
plt.savefig('DT vs Naive Account Value Over Time.png')


In [33]:
plt.rcParams['figure.figsize'] = (15,5)
ax = result[['PPO', '1/n']].plot()
plt.title('PPO vs Naive Account Value Over Time')
plt.xlabel('date')
plt.ylabel('Account Value')
plt.legend(['PPO', 'Naive'])
plt.grid(True)
plt.savefig('PPO vs Naive Account Value Over Time.png')



In [34]:
plt.rcParams['figure.figsize'] = (15, 5)
ax = result[['PPO', 'DT']].plot()
plt.title('PPO vs DT Account Value Over Time')
plt.xlabel('date')
plt.ylabel('Account Value')
plt.legend(['PPO', 'DT'])
plt.grid(True)
plt.savefig('PPO vs DT Account Value Over Time.png')


In [36]:
plt.rcParams['figure.figsize'] = (15, 5)
ax = result[['DT', '1/n', 'PPO']].plot()
plt.title('DT, Naive, and PPO Account Value Over Time')
plt.xlabel('date')
plt.ylabel('Account Value')
plt.legend(['DT', 'Naive', 'PPO'])
plt.grid(True)
plt.savefig('DT, Naive, and PPO Account Value Over Time.png')





In [53]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 2, figsize=(18, 12))

# First plot: DT vs Naive
# DT = blue, PPO = red, Naive = green
result[['DT', '1/n']].plot(ax=axs[0, 0], color=['blue', 'green'])
axs[0, 0].set_xlabel('Date', fontsize=16)
axs[0, 0].set_ylabel('Account Value', fontsize=16)
axs[0, 0].legend(['DT', 'Naive'], fontsize=14)
axs[0, 0].grid(True)

# Second plot: PPO vs Naive
result[['PPO', '1/n']].plot(ax=axs[0, 1], color=['red', 'green'])
axs[0, 1].set_xlabel('Date', fontsize=16)
axs[0, 1].set_ylabel('Account Value', fontsize=16)
axs[0, 1].legend(['PPO', 'Naive'], fontsize=14)
axs[0, 1].grid(True)

# Third plot: PPO vs DT
result[['PPO', 'DT']].plot(ax=axs[1, 0], color=['red', 'blue'])
axs[1, 0].set_xlabel('Date', fontsize=16)
axs[1, 0].set_ylabel('Account Value', fontsize=16)
axs[1, 0].legend(['PPO', 'DT'], fontsize=14)
axs[1, 0].grid(True)

# Fourth plot: DT, Naive, and PPO
result[['DT', '1/n', 'PPO']].plot(ax=axs[1, 1], color=['blue', 'green', 'red'])
axs[1, 1].set_xlabel('Date', fontsize=16)
axs[1, 1].set_ylabel('Account Value', fontsize=16)
axs[1, 1].legend(['DT', 'Naive', 'PPO'], fontsize=14)
axs[1, 1].grid(True)

plt.tight_layout()
plt.savefig('plots/results.png')
plt.show()


C:\Users\drebi\AppData\Local\Temp\ipykernel_31132\1193238744.py:36: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [44]:
from pyfolio import timeseries
DLR_strat = convert_daily_return_to_pyfolio_ts(result['DT'])
perf_func = timeseries.perf_stats
perf_stats_all = perf_func(returns=DLR_strat,
                       factor_returns=DLR_strat,
                        positions=None,
                        transactions=None,
                        turnover_denom="AGB")

KeyError: 'date'

In [81]:
df_daily_return_dt = result[['DT']]
df_daily_return_dt = df_daily_return_dt.reset_index()
df_daily_return_dt['daily_return'] = df_daily_return_dt['DT'].pct_change()
df_daily_return_dt.drop(columns=['DT'], inplace=True)
df_daily_return_dt.head()

DLR_strat = convert_daily_return_to_pyfolio_ts(df_daily_return_dt)
perf_func = timeseries.perf_stats

perf_stats_all = perf_func(returns=DLR_strat,
                           factor_returns=DLR_strat,
                           positions=None,
                           transactions=None,
                           turnover_denom='AGB')

perf_stats_all

C:\Users\drebi\AppData\Local\Temp\ipykernel_31132\4009208615.py:3: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_daily_return_dt['daily_return'] = df_daily_return_dt['DT'].pct_change()


Annual return          0.148791
Cumulative returns     0.288143
Annual volatility      0.156419
Sharpe ratio           0.966856
Calmar ratio           0.991016
Stability              0.737634
Max drawdown          -0.150140
Omega ratio            1.177531
Sortino ratio          1.469489
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.164945
Daily value at risk   -0.019107
Alpha                  0.000000
Beta                   1.000000
dtype: float64

In [82]:
df_daily_return_naive = result[['1/n']]
df_daily_return_naive = df_daily_return_naive.reset_index()
df_daily_return_naive['daily_return'] = df_daily_return_naive['1/n'].pct_change()
df_daily_return_naive.drop(columns=['1/n'], inplace=True)
df_daily_return_naive.head()

DLR_strat = convert_daily_return_to_pyfolio_ts(df_daily_return_naive)
perf_func = timeseries.perf_stats

perf_stats_all = perf_func(returns=DLR_strat,
                           factor_returns=DLR_strat,
                           positions=None,
                           transactions=None,
                           turnover_denom='AGB')

perf_stats_all

C:\Users\drebi\AppData\Local\Temp\ipykernel_31132\1805422301.py:3: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_daily_return_naive['daily_return'] = df_daily_return_naive['1/n'].pct_change()


Annual return          0.160889
Cumulative returns     0.313012
Annual volatility      0.155631
Sharpe ratio           1.038450
Calmar ratio           1.035026
Stability              0.769331
Max drawdown          -0.155444
Omega ratio            1.191676
Sortino ratio          1.584771
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.120236
Daily value at risk   -0.018966
Alpha                  0.000000
Beta                   1.000000
dtype: float64

In [83]:
df_daily_return_ppo = result[['PPO']]
df_daily_return_ppo = df_daily_return_ppo.reset_index()
df_daily_return_ppo['daily_return'] = df_daily_return_ppo['PPO'].pct_change()
df_daily_return_ppo.drop(columns=['PPO'], inplace=True)
df_daily_return_ppo.head()

DLR_strat = convert_daily_return_to_pyfolio_ts(df_daily_return_ppo)
perf_func = timeseries.perf_stats

perf_stats_all = perf_func(returns=DLR_strat,
                           factor_returns=DLR_strat,
                           positions=None,
                           transactions=None,
                           turnover_denom='AGB')

perf_stats_all

C:\Users\drebi\AppData\Local\Temp\ipykernel_31132\2933789221.py:3: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_daily_return_ppo['daily_return'] = df_daily_return_ppo['PPO'].pct_change()


Annual return          0.249243
Cumulative returns     0.501133
Annual volatility      0.155000
Sharpe ratio           1.516675
Calmar ratio           1.558233
Stability              0.886410
Max drawdown          -0.159952
Omega ratio            1.289953
Sortino ratio          2.359944
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.247778
Daily value at risk   -0.018595
Alpha                  0.000000
Beta                   1.000000
dtype: float64